In [ ]:
import pandas as pd
from emutools.tex import StandardTexDoc
import arviz as az
from inputs.constants import ANALYSIS_START_DATE, ANALYSIS_END_DATE, SUPPLEMENT_PATH
from aust_covid.plotting import plot_key_outputs, plot_cdr_examples, plot_subvariant_props, plot_dispersion_examples
from aust_covid.calibration import get_priors, get_targets
from emutools.calibration import get_sampled_outputs
import plotly.graph_objects as go

In [ ]:
app_doc = StandardTexDoc(SUPPLEMENT_PATH, 'supplement', "Australia's 2023 Omicron Waves Supplement", 'austcovid')

In [ ]:
spaghetti = pd.read_csv('spaghetti.csv', header=[0, 1, 2], index_col=[0])
burnt_idata = az.from_netcdf('idata_for_spaghetti.nc')

In [ ]:
targets = get_targets(app_doc)

In [ ]:
def get_param_values(idata, chain, draw):
    return {i: float(j) for i, j in dict(burnt_idata.posterior.sel(chain=int(chain), draw=int(draw)).variables).items()}

In [ ]:
fig = go.Figure()
indicator_spaghetti = spaghetti['adult_seropos_prop']
for i in indicator_spaghetti.columns:
    print(get_param_values(burnt_idata, i[0], i[1]))
    run_values = indicator_spaghetti[i]
    fig.add_trace(go.Scatter(x=run_values.index, y=run_values))
fig